In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
data_path = 'final_integrated_data.csv'  # Modify this path to your actual data file location
data = pd.read_csv(data_path)

# Clean and prepare the data
data.dropna(subset=['Life_Expectancy'], inplace=True)
X = data[['Country', 'Age', 'Cigarettes_Per_Day']]
y = data['Life_Expectancy']

# Preprocessing for categorical data
categorical_features = ['Country']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with a RandomForestRegressor and parameter tuning
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

# Train and evaluate the model
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print("Best Model Parameters:", grid_search.best_params_)
print("Mean Squared Error:", mse_best)
print("R2 Score:", r2_best)


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Model Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 50}
Mean Squared Error: 9.18642222546393
R2 Score: -0.03763748906611264


In [2]:
import pickle

# Save the model to a file
model_filename = 'final_life_expectancy_model.pkl'
pickle.dump(best_model, open(model_filename, 'wb'))
